In [ ]:
!pip install pandas

# Overtime Calculator

Set-up the following variables to reflect your working hours.

In [ ]:
UTC_DIFF = 1
WORKING_HOURS = [7,8,9,10,11,12,13,14,15,16,17,18]
WORKING_DAYS = [0,1,2,3,4] # 0-> Monday
EXCEPT_BETWEEN = [] # Use this for extraordinary situations. Ex, you had a week long extra vacation so you worked more in this time for paying it back etc.
LOCAL_WORKING_HOURS = list(map(lambda x: x+UTC_DIFF, WORKING_HOURS))

In [ ]:
PATH = "/Users/gokaykucuk/concular/concore-django"

In [ ]:
!git --git-dir=$PATH/.git log --reflog --pretty=format:%aI_SEPERATOR_%ae > log.csv

In [ ]:
open("log_with_headers.csv", "w").write("time_SEPERATOR_email\n" + open("log.csv").read())

In [ ]:
import pandas as pd

In [ ]:
git_log_df = pd.read_csv("log_with_headers.csv", sep="_SEPERATOR_")

In [ ]:
git_log_df['time']=pd.to_datetime(git_log_df['time'], utc=True)
git_log_df.set_index("time")

# Setup Working Hours

Setup the working hours and days. Also setup the timezone difference from UTC with your local timezone.

In [ ]:
git_log_df['day_of_week'] = git_log_df['time'].dt.dayofweek
git_log_df['hour_of_day'] = git_log_df['time'].dt.hour
git_log_df['date'] = git_log_df['time'].dt.date

In [ ]:
git_log_df

In [ ]:
git_log_df['overtime_day'] = ~git_log_df['day_of_week'].isin(WORKING_DAYS)
git_log_df['overtime_hour'] = ~git_log_df['hour_of_day'].isin(WORKING_HOURS)

In [ ]:
email_filter = "gokay"

In [ ]:
personal_timesheet_df = git_log_df[git_log_df["email"].str.contains(email_filter)]

In [ ]:
personal_timesheet_df

# Calculate Overtime for Off Days

In [ ]:
overtime_off_days_df = personal_timesheet_df[personal_timesheet_df["overtime_day"] == True]

In [ ]:
overtime_days_off_series = overtime_off_days_df.groupby("date", group_keys=True).apply(lambda x: (x["hour_of_day"].max() - x["hour_of_day"].min()+1))

In [ ]:
overtime_days_off_series

In [ ]:
overtime_days_off_series.sum()

# Calculate Overtime for Work Days

In [ ]:
overtime_work_days_df = overtime_days_df = personal_timesheet_df[(personal_timesheet_df["overtime_day"] == False) & (personal_timesheet_df["overtime_hour"] == True)]

In [ ]:
overtime_work_days_df

In [ ]:
rough_calculation_series = overtime_work_days_df.groupby("date", group_keys=True).apply(lambda x: (x["hour_of_day"].max() - x["hour_of_day"].min()+1))

In [ ]:
rough_calculation_series = rough_calculation_series[rough_calculation_series > 10] - 10

In [ ]:
rough_calculation_series.sum()